In [2]:
# 데이터 분석에 사용할 라이브러리
import pandas as pd
import numpy as np 
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns

# error 창 안 보이게 해줌
import logging
logging.getLogger('matplotlib.font_manager').setLevel(logging.ERROR)

# 가용폰트 확인
for font in mpl.font_manager.findSystemFonts():
    # print(mpl.font_manager.FontProperties(fname=font).get_name())
    mpl.font_manager.fontManager.addfont(font)

# Set font family globally
mpl.rcParams['font.family'] = 'NanumBarunGothic'# 폰트명
print(mpl.rcParams['font.family'])

['NanumBarunGothic']


In [3]:
import easydict
args = easydict.EasyDict()

In [4]:
default_path = 'C:/titanic_datas/'
train = default_path + 'train.csv'
test = default_path + 'test.csv'
default_submission = default_path + 'submission.csv'

submission_csv = default_path + 'result/solution.csv'
save_results = default_path + 'result/model_results.json'

random_state = 21
results = []

In [5]:
DATA_PATH = 'C:/titanic_datas/'

df = pd.read_csv(DATA_PATH+'train.csv')
df.head()

,passengerid,survived,pclass,name,gender,age,sibsp,parch,ticket,fare,cabin,embarked
0,0,0,2,"Wheeler, Mr. Edwin Frederick""""",male,NaN,0,0,SC/PARIS 2159,12.8750,NaN,S
1,1,0,3,"Henry, Miss. Delia",female,NaN,0,0,382649,7.7500,NaN,Q
2,2,1,1,"Hays, Mrs. Charles Melville (Clara Jennings Gr...",female,52.0,1,1,12749,93.5000,B69,S
3,3,1,3,"Andersson, Mr. August Edvard (""Wennerstrom"")",male,27.0,0,0,350043,7.7958,NaN,S
4,4,0,2,"Hold, Mr. Stephen",male,44.0,1,0,26707,26.0000,NaN,S


In [6]:
origin_train = pd.read_csv(train)
origin_test = pd.read_csv(test)

origin_train.shape, origin_test.shape

((916, 12), (393, 11))

In [7]:
pd.read_csv(default_submission).shape

(393, 2)

In [8]:
origin_train.columns

Index(['passengerid', 'survived', 'pclass', 'name', 'gender', 'age', 'sibsp',
       'parch', 'ticket', 'fare', 'cabin', 'embarked'],
      dtype='object')

- features
- 귀족들이 많다. -> name에서 추출
- cabin -> 세월호 때 호실 찾는 거, 기록이 안 된 사람들은 사망했을 가능성이 높다.
- age -> 결측치 넣기 전 분포로 확인함, 결측치를 무엇으로 채울 지 결정

In [15]:
dict_designation = {
    'Mr.': '남성',
    'Master.': '남성',
    'Sir.': '남성',
    'Miss.': '미혼 여성',
    'Mrs.': '기혼 여성',
    'Ms.': '미혼/기혼 여성',
    'Lady.': '숙녀',
    'Mlle.': '아가씨',
    # 직업
    'Dr.': '의사',
    'Rev.': '목사',
    'Major.': '계급',
    'Don.': '교수',
    'Col.': '군인',
    'Capt.': '군인',
    # 귀족
    'Mme.': '영부인',
    'Countess.': '백작부인',
    'Jonkheer.': '귀족'
}

def add_designation(name): # 호칭 함수
  designation = "unknown"
  for key in dict_designation.keys():
    if key in name:
      designation = key
      break
  return designation

origin_train['designation'] = origin_train['name'].map(lambda x: add_designation(x))
origin_test['designation'] = origin_test['name'].map(lambda x: add_designation(x))

origin_train.designation.value_counts()

Mr.          528
Miss.        185
Mrs.         138
Master.       42
Rev.           6
Dr.            5
Col.           3
Major.         2
Don.           1
Ms.            1
Jonkheer.      1
Mlle.          1
Countess.      1
Capt.          1
Sir.           1
Name: designation, dtype: int64

In [16]:
pd.crosstab(origin_train.designation,origin_train.gender).T.style.background_gradient(cmap='summer_r') 
#Checking the Initials with the Gender

designation,Capt.,Col.,Countess.,Don.,Dr.,Jonkheer.,Major.,Master.,Miss.,Mlle.,Mr.,Mrs.,Ms.,Rev.,Sir.
gender,,,,,,,,,,,,,,,
female,0,0,1,0,1,0,0,0,185,1,0,138,1,0,0
male,1,3,0,1,4,1,2,42,0,0,528,0,0,6,1


In [19]:
origin_train.cabin.isnull().sum()

718